<a href="https://colab.research.google.com/github/MichaelGuaca/Proyecto-IA/blob/main/05_modelo_soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importar librerías necesarias
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
import xgboost as xgb


In [ ]:
#IMPORTAR DATOS DE KAGGLE A COLAB
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242

 70% 14.0M/20.1M [00:00<00:00, 142MB/s]
100% 20.1M/20.1M [00:00<00:00, 159MB/s]


In [ ]:
#DESCOMPRIMIR ARCHIVO
!unzip udea-ai4eng-20242.zip

Archive:  udea-ai4eng-20242.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
#CARGAR DATOS TRAIN CON PANDAS
train_data = pd.read_csv("train.csv")

In [ ]:
#Explorar datos
print(train_data.info())
print(train_data['RENDIMIENTO_GLOBAL'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 12 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   ID                              692500 non-null  int64 
 1   PERIODO                         692500 non-null  int64 
 2   ESTU_PRGM_ACADEMICO             692500 non-null  object
 3   ESTU_PRGM_DEPARTAMENTO          692500 non-null  object
 4   ESTU_VALORMATRICULAUNIVERSIDAD  686213 non-null  object
 5   ESTU_HORASSEMANATRABAJA         661643 non-null  object
 6   FAMI_ESTRATOVIVIENDA            660363 non-null  object
 7   FAMI_TIENEINTERNET              665871 non-null  object
 8   FAMI_EDUCACIONPADRE             669322 non-null  object
 9   FAMI_EDUCACIONMADRE             668836 non-null  object
 10  ESTU_PAGOMATRICULAPROPIO        686002 non-null  object
 11  RENDIMIENTO_GLOBAL              692500 non-null  object
dtypes: int64(2), object(10)
memory

In [ ]:
#Preprocesamiento de datos
#Manejar valores faltantes (rellenando con una categoría específica para columnas categóricas)
for col in train_data.columns:
    if train_data[col].dtype == 'object':
        train_data[col].fillna('Desconocido', inplace=True)
    else:
        train_data[col].fillna(train_data[col].median(), inplace=True)

<ipython-input-32-73dd2afa7a95>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data[col].fillna(train_data[col].median(), inplace=True)


In [ ]:
#Codificar variables categóricas usando LabelEncoder
categorical_cols = train_data.select_dtypes(include='object').columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    train_data[col] = le.fit_transform(train_data[col])
    label_encoders[col] = le

In [ ]:
#Separar características (X) y objetivo (y)
X = train_data.drop(['RENDIMIENTO_GLOBAL', 'ID', 'PERIODO'], axis=1)
y = train_data['RENDIMIENTO_GLOBAL']

In [ ]:
#Dividir los datos en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Entrenar modelo XGBoost
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb_model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [02:56:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [ ]:
# Evaluar el modelo
xgb_pred = xgb_model.predict(X_val)
xgb_accuracy = accuracy_score(y_val, xgb_pred)
print(f"XGBoost Accuracy: {xgb_accuracy}")
print(classification_report(y_val, xgb_pred, target_names=label_encoders["RENDIMIENTO_GLOBAL"].classes_))


XGBoost Accuracy: 0.43094584837545125
              precision    recall  f1-score   support

        alto       0.55      0.63      0.59     35165
        bajo       0.46      0.55      0.50     34573
  medio-alto       0.33      0.27      0.29     34259
  medio-bajo       0.33      0.27      0.29     34503

    accuracy                           0.43    138500
   macro avg       0.41      0.43      0.42    138500
weighted avg       0.42      0.43      0.42    138500



In [ ]:
submission = pd.DataFrame([train_data['ID'] , pd.Series(xgb_pred, name="RENDIMIENTO_GLOBAL")]).T
submission.head()

,ID,RENDIMIENTO_GLOBAL
0,904256.0,3.0
1,645256.0,0.0
2,308367.0,2.0
3,470353.0,0.0
4,989032.0,1.0


from matplotlib import pyplot as plt
_df_0['ID'].plot(kind='hist', bins=20, title='ID')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['RENDIMIENTO_GLOBAL'].plot(kind='hist', bins=20, title='RENDIMIENTO_GLOBAL')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2.plot(kind='scatter', x='ID', y='RENDIMIENTO_GLOBAL', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['ID'].plot(kind='line', figsize=(8, 4), title='ID')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_4['RENDIMIENTO_GLOBAL'].plot(kind='line', figsize=(8, 4), title='RENDIMIENTO_GLOBAL')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
submission.to_csv("Submission_File", index=False)

In [ ]:
# Crear la carpeta .kaggle y mover el archivo json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Cambiar los permisos del archivo
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions submit -c udea-ai4eng-20242 -f /content/submission_File.csv -m "Primera Entrega"

100% 4.50M/4.50M [00:00<00:00, 9.66MB/s]
Successfully submitted to UDEA/ai4eng 20242 - Pruebas Saber Pro Colombia